In [1]:
import pandas as pd

df=pd.read_csv(r"F:\MOOO\AI_AMIT_DEPI\projects\ML project\archive\T.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [3]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [4]:
df.drop(["Loan_ID"],axis=1,inplace=True)

In [7]:
df["TotalIncome"]=df["ApplicantIncome"]+df["CoapplicantIncome"]
df.drop(["ApplicantIncome","CoapplicantIncome"],axis=1,inplace=True)

In [8]:
df

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome
0,Male,No,0,Graduate,No,NaN,360.0,1.0,Urban,Y,5849.0
1,Male,Yes,1,Graduate,No,128.0,360.0,1.0,Rural,N,6091.0
2,Male,Yes,0,Graduate,Yes,66.0,360.0,1.0,Urban,Y,3000.0
3,Male,Yes,0,Not Graduate,No,120.0,360.0,1.0,Urban,Y,4941.0
4,Male,No,0,Graduate,No,141.0,360.0,1.0,Urban,Y,6000.0
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,71.0,360.0,1.0,Rural,Y,2900.0
610,Male,Yes,3+,Graduate,No,40.0,180.0,1.0,Rural,Y,4106.0
611,Male,Yes,1,Graduate,No,253.0,360.0,1.0,Urban,Y,8312.0
612,Male,Yes,2,Graduate,No,187.0,360.0,1.0,Urban,Y,7583.0


In [9]:
from datasist.structdata import detect_outliers
dirty_cols =df.select_dtypes(['int64','float64'])
dirty_idx=detect_outliers(data=df,n=0,features=dirty_cols)
df.drop(dirty_idx,inplace=True)

In [10]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy='mean')
dirty_cols=dirty_cols.columns
df[dirty_cols]=imputer.fit_transform(df[dirty_cols])

In [11]:
imputer=SimpleImputer(strategy="most_frequent")
dirty_cols=df.select_dtypes(['object'])
dirty_cols=dirty_cols.columns
df[dirty_cols]=imputer.fit_transform(df[dirty_cols])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 564 entries, 0 to 613
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender            564 non-null    object 
 1   Married           564 non-null    object 
 2   Dependents        564 non-null    object 
 3   Education         564 non-null    object 
 4   Self_Employed     564 non-null    object 
 5   LoanAmount        564 non-null    float64
 6   Loan_Amount_Term  564 non-null    float64
 7   Credit_History    564 non-null    float64
 8   Property_Area     564 non-null    object 
 9   Loan_Status       564 non-null    object 
 10  TotalIncome       564 non-null    float64
dtypes: float64(4), object(7)
memory usage: 52.9+ KB


In [13]:
df = pd.get_dummies(df, columns=["Gender","Married","Education","Self_Employed","Loan_Status","Dependents","Property_Area"], drop_first=True)

In [14]:
from sklearn.model_selection import train_test_split
y=df["Loan_Status_Y"]
x=df.drop("Loan_Status_Y",axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [15]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols=["Loan_Amount_Term","LoanAmount","TotalIncome","Credit_History"]
scaler.fit(x_train[cols])


RobustScaler()

In [16]:
x_train=scaler.fit_transform(x_train[cols])
x_test=scaler.fit_transform(x_test[cols])

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [18]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(x_train, y_train)

print_score(tree_clf, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf, x_train, y_train, x_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
           False   True  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    118.0  276.0       1.0      394.0         394.0
_______________________________________________
Confusion Matrix: 
 [[118   0]
 [  0 276]]

Test Result:
Accuracy Score: 72.35%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.586957    0.774194  0.723529    0.680575      0.713617
recall      0.490909    0.834783  0.723529    0.662846      0.723529
f1-score    0.534653    0.803347  0.723529    0.669000      0.716417
support    55.000000  115.000000  0.723529  170.000000    170.000000
_______________________________________________
Conf

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"),
    "splitter":("best", "random"),
    "max_depth":(list(range(1, 20))),
    "min_samples_split":[2, 3, 4],
    "min_samples_leaf":list(range(1, 20)),
}


tree_clf = DecisionTreeClassifier(random_state=42)
tree_cv = GridSearchCV(tree_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)
tree_cv.fit(x_train, y_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

tree_clf_cv = DecisionTreeClassifier(**best_params)
tree_clf_cv.fit(x_train, y_train)
print_score(tree_clf_cv, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf_cv, x_train, y_train, x_test, y_test, train=False)

Fitting 3 folds for each of 4332 candidates, totalling 12996 fits
Best paramters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'})
Train Result:
Accuracy Score: 83.25%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.964286    0.810651  0.832487    0.887468      0.856663
recall       0.457627    0.992754  0.832487    0.725190      0.832487
f1-score     0.620690    0.892508  0.832487    0.756599      0.811101
support    118.000000  276.000000  0.832487  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 54  64]
 [  2 274]]

Test Result:
Accuracy Score: 80.00%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.888889    0.783217       0.8    0.836053      0.817405
recall      

In [20]:
from sklearn.linear_model import LogisticRegression

log=LogisticRegression()
reg=log.fit(x_train,y_train)
print_score(reg, x_train, y_train, x_test, y_test, train=True)
print_score(reg, x_train, y_train, x_test, y_test, train=False)

Train Result:
Accuracy Score: 82.49%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.945455    0.805310  0.824873    0.875382      0.847282
recall       0.440678    0.989130  0.824873    0.714904      0.824873
f1-score     0.601156    0.887805  0.824873    0.744480      0.801956
support    118.000000  276.000000  0.824873  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 52  66]
 [  3 273]]

Test Result:
Accuracy Score: 79.41%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.884615    0.777778  0.794118    0.831197      0.812343
recall      0.418182    0.973913  0.794118    0.696047      0.794118
f1-score    0.567901    0.864865  0.794118    0.716383      0.768788
support    55.000000  115.000000  0.794118  170.000000    170.00000

In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ["saga", "lbfgs", "liblinear"],  
    'l1_ratio': [0.1, 0.5, 0.9]  
}

grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

print_score(tree_clf, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf, x_train, y_train, x_test, y_test, train=False)

c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\l

Train Result:
Accuracy Score: 83.50%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.907692    0.820669  0.835025    0.864181      0.846732
recall       0.500000    0.978261  0.835025    0.739130      0.835025
f1-score     0.644809    0.892562  0.835025    0.768685      0.818362
support    118.000000  276.000000  0.835025  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 59  59]
 [  6 270]]

Test Result:
Accuracy Score: 80.00%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.838710    0.791367       0.8    0.815038      0.806684
recall      0.472727    0.956522       0.8    0.714625      0.800000
f1-score    0.604651    0.866142       0.8    0.735396      0.781542
support    55.000000  115.000000       0.8  170.000000    170.00000

c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_l

In [26]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(x_train, y_train)

print_score(rf_clf, x_train, y_train, x_test, y_test, train=True)
print_score(rf_clf, x_train, y_train, x_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
           False   True  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    118.0  276.0       1.0      394.0         394.0
_______________________________________________
Confusion Matrix: 
 [[118   0]
 [  0 276]]

Test Result:
Accuracy Score: 78.24%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.750000    0.791045  0.782353    0.770522      0.777766
recall      0.490909    0.921739  0.782353    0.706324      0.782353
f1-score    0.593407    0.851406  0.782353    0.722406      0.767935
support    55.000000  115.000000  0.782353  170.000000    170.000000
_______________________________________________
Conf

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = RandomizedSearchCV(estimator=rf_clf, scoring='f1',param_distributions=random_grid, n_iter=100, cv=3,
                               verbose=2, random_state=42, n_jobs=-1)

rf_cv.fit(x_train, y_train)
rf_best_params = rf_cv.best_params_
print(f"Best paramters: {rf_best_params})")

rf_clf = RandomForestClassifier(**rf_best_params)
rf_clf.fit(x_train, y_train)

print_score(rf_clf, x_train, y_train, x_test, y_test, train=True)
print_score(rf_clf, x_train, y_train, x_test, y_test, train=False)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best paramters: {'n_estimators': 800, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 110, 'bootstrap': True})
Train Result:
Accuracy Score: 85.28%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.916667    0.838509  0.852792    0.877588      0.861917
recall       0.559322    0.978261  0.852792    0.768791      0.852792
f1-score     0.694737    0.903010  0.852792    0.798873      0.840634
support    118.000000  276.000000  0.852792  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 66  52]
 [  6 270]]

Test Result:
Accuracy Score: 80.00%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.838710    0.791367       0.8    0.815038      0.806684
recall      0.47272